In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
train_df = pd.read_csv('train_preprocessing.csv')
test_df = pd.read_csv('test_preprocessing.csv')

In [9]:
train_df

,Unnamed: 0,keyword,location,text,target
0,0,None,None,Our Deeds are the Reason of this earthquake Ma...,1
1,1,None,None,Forest fire near La Ronge Sask Canada,1
2,2,None,None,All residents asked to shelter in place are be...,1
3,3,None,None,13000 people receive wildfires evacuation orde...,1
4,4,None,None,Just got sent this photo from Ruby Alaska as s...,1
...,...,...,...,...,...
7498,7604,None,None,WorldNews Fallen powerlines on Glink tram UPDA...,1
7499,7605,None,None,on the flip side Im at Walmart and there is a ...,1
7500,7606,None,None,Suicide bomber kills 15 in Saudi security site...,1
7501,7608,None,None,Two giant cranes holding a bridge collapse int...,1


In [3]:
x = train_df['text']
y = train_df['target']
x1 = test_df['text']

In [4]:
train_texts, test_texts, train_labels, test_labels = train_test_split(x.values, y.values, test_size=.20, random_state=42, stratify=y)


In [5]:
from transformers import RobertaTokenizer
from transformers import TFRobertaForSequenceClassification
import tensorflow as tf


2023-06-27 15:44:47.540848: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-27 15:44:47.542266: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-27 15:44:47.571289: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-27 15:44:47.572020: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-27 15:44:48.140517: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [6]:
# Khởi tạo mô hình
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [7]:
#Encode train input
encoded_inputs = tokenizer(train_texts.tolist(), padding=True, truncation=True, return_tensors='tf')
labels = tf.constant(train_labels.tolist())

In [8]:
# Define the training parameters
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

In [11]:
# Compile and train the model
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
model.fit(encoded_inputs.input_ids, labels, epochs=2, batch_size=32)

Epoch 1/2
188/188 [==============================] - 930s 5s/step - loss: 0.4975 - accuracy: 0.7642
Epoch 2/2
188/188 [==============================] - 893s 5s/step - loss: 0.3833 - accuracy: 0.8426


In [16]:
test_labels

<tf.Tensor: shape=(1501,), dtype=int32, numpy=array([0, 1, 0, ..., 1, 1, 1], dtype=int32)>

In [17]:
#Encode test input
test_inputs = tokenizer(test_texts.tolist(), padding=True, truncation=True, return_tensors='tf')
#labels = tf.constant(test_labels.tolist())


model.evaluate(
    x=test_inputs.input_ids,
    y=test_labels,
    batch_size=32
)

'''
#Evaluate the model
predictions = model.predict(test_inputs.input_ids)
predicted_labels = eval_predictions = tf.argmax(tf.nn.softmax(predictions, axis=1), axis=1).numpy()

accuracy = accuracy_score(test_labels, predicted_labels)
print("Accuracy:", accuracy)
'''

47/47 [==============================] - 41s 837ms/step - loss: 0.3968 - accuracy: 0.8434


'\n#Evaluate the model\npredictions = model.predict(test_inputs.input_ids)\npredicted_labels = eval_predictions = tf.argmax(tf.nn.softmax(predictions, axis=1), axis=1).numpy()\n\naccuracy = accuracy_score(test_labels, predicted_labels)\nprint("Accuracy:", accuracy)\n'

In [18]:
#Encode predict input
predict_inputs = tokenizer(x1.tolist(), padding=True, truncation=True, return_tensors='tf')
# Make predictions
predictions = model.predict(predict_inputs.input_ids)

# Extract predicted labels
predicted_labels = tf.argmax(predictions.logits, axis=1).numpy()

102/102 [==============================] - 122s 1s/step


In [ ]:
predicted_labels

In [ ]:
submission = pd.DataFrame({'id': test_df['id'], 'target': predicted_labels})
submission.to_csv('submission.csv', index=False)